## 1. 환경설정

In [1]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

## 2. 모델로드

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-Small-24B-Instruct-2501", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/164k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # [변경] embed_tokens랑 lm_head 추가함
    lora_alpha = 32,
    lora_dropout = 0.05, 
    bias = "none",   
   
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora=False,  
    loftq_config=None, 
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Offloading input_embeddings to disk to save VRAM


/usr/local/lib/python3.11/dist-packages/unsloth/models/_utils.py:760: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filename, map_location = "cpu", 

Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.2.15 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


## 3. 파일 로드

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import drive
import zipfile
import os
# ZIP 파일 경로 (Google Drive에 있는 파일 경로 입력)
zip_file_path = '/content/drive/MyDrive/한글리뷰/open.zip'

# 압축 해제 경로 설정
extract_to_path = '/content/extracted_files'

# 압축 해제 실행
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"ZIP 파일이 성공적으로 {extract_to_path} 경로에 압축 해제되었습니다.")

ZIP 파일이 성공적으로 /content/extracted_files 경로에 압축 해제되었습니다.


## 4. 데이터셋 생성

In [5]:
import pandas as pd
train = pd.read_csv('./extracted_files/train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./extracted_files/test.csv', encoding = 'utf-8-sig')

In [7]:
def llama_formatting(examples):
  inputs = []
  outputs =[]
  for inp, out in zip(examples['input'], examples['output']):
    inputs.append(inp)
    outputs.append(out)

  return {"input" : inputs, "output": outputs}


llama_dataset = llama_formatting(train)

In [8]:
from datasets import Dataset
# Dataset 객체로 변환

llama_dataset_prompt_map = Dataset.from_dict(llama_dataset)
llama_dataset_prompt_map

Dataset({
    features: ['input', 'output'],
    num_rows: 11263
})

In [9]:
from datasets import load_dataset

# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """You are a Korean language expert. Convert the given obfuscated Korean sentences into clear and understandable Korean sentences.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["input"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }


# "teddylee777/QA-Dataset-mini" 데이터셋을 불러옵니다. 훈련 데이터만 사용합니다.
dataset = llama_dataset_prompt_map

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/11263 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 11263
})

## 5. 모델 설정

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4096,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps=5,
        do_eval=False,
        max_steps = 1500,
        learning_rate = 2e-4,
        embedding_learning_rate = 2e-5,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay =0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=8):   0%|          | 0/11263 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=8):   0%|          | 0/11263 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=8):   0%|          | 0/11263 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=8):   0%|          | 0/11263 [00:00<?, ? examples/s]

## 6. 학습

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,263 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 1,500
 "-____-"     Number of trainable parameters = 1,434,583,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: schaewon324 (schaewon324-private) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,3.417400
20,2.949600
30,2.722500
40,2.592200
50,2.401500
60,2.404900
70,2.370700
80,2.291000
90,2.224800
100,2.245100


## 7. 샘플로 추론 결과 확인해보기

In [16]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "쑴엉창 뮬 콴뤼 앉 하쉬는 트소. 엑 면쥐 퉁퉁 펄렘 듕튱 터단닉교 팝붸뀨창 않넥 되쥐 뷔륀넥갸 낱써 꾸역질잎 냐떨락교욤. 환뿡긱예 뻘뤠 둘 먀릭량 면쥐 뎡얼뤼엣 먀꼇셔 뜰얹놔톰 환긱꺄 얀 퇴교, 쳔쟝엥 꼼팡위 ㅜㅜ 확짱싫 쉴릭꼰녜돈 균텔쿤뎨 곪빵잎... 팝벡꾸 신쩡했는테 뙈짇 목살엑섯 뉼린낵캄 ㅜㅜㅜㅜ 멱딱갸 속 얀 죠햐저설 골킷 담 벗렀엇오. 솥쎄찌... 살댜 샬닥 윌렇궤 퍽뼈깍교 맏엎눈 솔쉐췻 쳐욺인넵오. 까띠 낮욘 빤쨘운 수텐 언형 쿱씩판에 낌칠랗 쿤냥 챘씩 빤쨘 죠굶쉭 떠려쳐 있섰는텟 얹쳬 뼈닮았눈짓토 묠룰 판짠.... 씰릴꼰 텁갤로 떱뻣셔 굳탯료 쥬쒸쿄  쿨뤽곱 물논뤼햐코 났왓는떼 쩌녘예 핏뿍카 웨 윕렇께 ㅜ 갼쥐럽던질. 냉붇는 구냠먈 캑꿎한테 넹쟝꼬눈 냉팡또 짤 얀 퉤교 청쑈 한낭됴 ㅠ 얀 뒈엶 잇우쉰 트소. 앝 삑윙 쌍헤오. 똔됴 야캅코 옙푸계 취엮놓흣설놓코 외 관뤼를 잎렇계 하쉰눈쥐... 놂쳔땅토 쉰쩡헷섯 타 즐끼러코 켸횐깡교 갗엊는텟 줴때롬 쎄쩍군 핫쒸눈치 읠쉼위 뒈엌 왠쥐 칩찝해썲 쿠냥 난왔눼욕. 첵꺄치 내섰긺램 염청 관뤼갸 짤됐곗견님 항꾜 귄때향꼽 갓눈뒈 따씬 얀 옭쿄 십얹오. 염긷써 멂꾼 교끼 템뮨에 ㅠ 쇽 않 좋핥적셔 닿뷴간 된칡곡길 멱코 십짖까 얀눼욜. 슈엉짱 묾 윕웨 떨탄닌는 뻘렐량 면칟갸 켸쇽 쌩꺅냐옮. 팎펙쿠쟝예 빼여 잇는 되취 냄쌔 땟문웨 잉 창솖만 뗘욜륌면 쟈똥읕롯 된찌 눌린냄갸 언샹위 됩닛댜. 쥔씸 큐역짊 낟는 켱헒 ㅠㅠ 좋궤 숟코 쉬뻤는뎨 윈쌩 콴립갸 넒뮤 엉먕친챵윌랄 칙끔토 넣뮤 휨튿려요.. 앓후..[/INST]",  # 지시사항
            "" # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


max_length = model.config.max_position_embeddings  # 모델의 최대 토큰 길이
inputs["input_ids"] = inputs["input_ids"][:, :max_length]
inputs["attention_mask"] = inputs["attention_mask"][:, :max_length]


text_streamer = TextStreamer(tokenizer)
result_sentence = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=2092,  # 최대 생성 토큰 수를 설정합니다.
    use_cache=True ,
    #stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)
import re

# ✅ 모델 응답 디코딩
decoded_output = tokenizer.batch_decode(result_sentence, skip_special_tokens=True)[0]

decoded_output


<s>You are a Korean language expert. Convert the given obfuscated Korean sentences into clear and understandable Korean sentences.

### Instruction:
쑴엉창 뮬 콴뤼 앉 하쉬는 트소. 엑 면쥐 퉁퉁 펄렘 듕튱 터단닉교 팝붸뀨창 않넥 되쥐 뷔륀넥갸 낱써 꾸역질잎 냐떨락교욤. 환뿡긱예 뻘뤠 둘 먀릭량 면쥐 뎡얼뤼엣 먀꼇셔 뜰얹놔톰 환긱꺄 얀 퇴교, 쳔쟝엥 꼼팡위 ㅜㅜ 확짱싫 쉴릭꼰녜돈 균텔쿤뎨 곪빵잎... 팝벡꾸 신쩡했는테 뙈짇 목살엑섯 뉼린낵캄 ㅜㅜㅜㅜ 멱딱갸 속 얀 죠햐저설 골킷 담 벗렀엇오. 솥쎄찌... 살댜 샬닥 윌렇궤 퍽뼈깍교 맏엎눈 솔쉐췻 쳐욺인넵오. 까띠 낮욘 빤쨘운 수텐 언형 쿱씩판에 낌칠랗 쿤냥 챘씩 빤쨘 죠굶쉭 떠려쳐 있섰는텟 얹쳬 뼈닮았눈짓토 묠룰 판짠.... 씰릴꼰 텁갤로 떱뻣셔 굳탯료 쥬쒸쿄  쿨뤽곱 물논뤼햐코 났왓는떼 쩌녘예 핏뿍카 웨 윕렇께 ㅜ 갼쥐럽던질. 냉붇는 구냠먈 캑꿎한테 넹쟝꼬눈 냉팡또 짤 얀 퉤교 청쑈 한낭됴 ㅠ 얀 뒈엶 잇우쉰 트소. 앝 삑윙 쌍헤오. 똔됴 야캅코 옙푸계 취엮놓흣설놓코 외 관뤼를 잎렇계 하쉰눈쥐... 놂쳔땅토 쉰쩡헷섯 타 즐끼러코 켸횐깡교 갗엊는텟 줴때롬 쎄쩍군 핫쒸눈치 읠쉼위 뒈엌 왠쥐 칩찝해썲 쿠냥 난왔눼욕. 첵꺄치 내섰긺램 염청 관뤼갸 짤됐곗견님 항꾜 귄때향꼽 갓눈뒈 따씬 얀 옭쿄 십얹오. 염긷써 멂꾼 교끼 템뮨에 ㅠ 쇽 않 좋핥적셔 닿뷴간 된칡곡길 멱코 십짖까 얀눼욜. 슈엉짱 묾 윕웨 떨탄닌는 뻘렐량 면칟갸 켸쇽 쌩꺅냐옮. 팎펙쿠쟝예 빼여 잇는 되취 냄쌔 땟문웨 잉 창솖만 뗘욜륌면 쟈똥읕롯 된찌 눌린냄갸 언샹위 됩닛댜. 쥔씸 큐역짊 낟는 켱헒 ㅠㅠ 좋궤 숟코 쉬뻤는뎨 윈쌩 콴립갸 넒뮤 엉먕친챵윌랄 칙끔토 넣뮤 휨튿려요.. 앓후..[/INST]

### Response:
수영장 물 관리 안 하시는 듯요. 애 먼지 둥둥 벌레 둥둥 떠다니고 바베큐장 안에 돼지 비린내가 나서 구역질이 나더라고요

'You are a Korean language expert. Convert the given obfuscated Korean sentences into clear and understandable Korean sentences.\n\n### Instruction:\n쑴엉창 뮬 콴뤼 앉 하쉬는 트소. 엑 면쥐 퉁퉁 펄렘 듕튱 터단닉교 팝붸뀨창 않넥 되쥐 뷔륀넥갸 낱써 꾸역질잎 냐떨락교욤. 환뿡긱예 뻘뤠 둘 먀릭량 면쥐 뎡얼뤼엣 먀꼇셔 뜰얹놔톰 환긱꺄 얀 퇴교, 쳔쟝엥 꼼팡위 ㅜㅜ 확짱싫 쉴릭꼰녜돈 균텔쿤뎨 곪빵잎... 팝벡꾸 신쩡했는테 뙈짇 목살엑섯 뉼린낵캄 ㅜㅜㅜㅜ 멱딱갸 속 얀 죠햐저설 골킷 담 벗렀엇오. 솥쎄찌... 살댜 샬닥 윌렇궤 퍽뼈깍교 맏엎눈 솔쉐췻 쳐욺인넵오. 까띠 낮욘 빤쨘운 수텐 언형 쿱씩판에 낌칠랗 쿤냥 챘씩 빤쨘 죠굶쉭 떠려쳐 있섰는텟 얹쳬 뼈닮았눈짓토 묠룰 판짠.... 씰릴꼰 텁갤로 떱뻣셔 굳탯료 쥬쒸쿄  쿨뤽곱 물논뤼햐코 났왓는떼 쩌녘예 핏뿍카 웨 윕렇께 ㅜ 갼쥐럽던질. 냉붇는 구냠먈 캑꿎한테 넹쟝꼬눈 냉팡또 짤 얀 퉤교 청쑈 한낭됴 ㅠ 얀 뒈엶 잇우쉰 트소. 앝 삑윙 쌍헤오. 똔됴 야캅코 옙푸계 취엮놓흣설놓코 외 관뤼를 잎렇계 하쉰눈쥐... 놂쳔땅토 쉰쩡헷섯 타 즐끼러코 켸횐깡교 갗엊는텟 줴때롬 쎄쩍군 핫쒸눈치 읠쉼위 뒈엌 왠쥐 칩찝해썲 쿠냥 난왔눼욕. 첵꺄치 내섰긺램 염청 관뤼갸 짤됐곗견님 항꾜 귄때향꼽 갓눈뒈 따씬 얀 옭쿄 십얹오. 염긷써 멂꾼 교끼 템뮨에 ㅠ 쇽 않 좋핥적셔 닿뷴간 된칡곡길 멱코 십짖까 얀눼욜. 슈엉짱 묾 윕웨 떨탄닌는 뻘렐량 면칟갸 켸쇽 쌩꺅냐옮. 팎펙쿠쟝예 빼여 잇는 되취 냄쌔 땟문웨 잉 창솖만 뗘욜륌면 쟈똥읕롯 된찌 눌린냄갸 언샹위 됩닛댜. 쥔씸 큐역짊 낟는 켱헒 ㅠㅠ 좋궤 숟코 쉬뻤는뎨 윈쌩 콴립갸 넒뮤 엉먕친챵윌랄 칙끔토 넣뮤 휨튿려요.. 앓후..\n\n### Response:\n수영장 물 관리 안 하시는 듯요. 애 먼지 둥둥 벌레 둥둥 떠다니고 바베큐장 안에 돼지 비린내가 나서 구역질이 나더라고요. 환

## 8. 모델 저장

In [14]:
# 모델 및 토크나이저 로컬 저장
output_dir = "/content/drive/MyDrive/한글리뷰/20250223_cpt_2-8_1500steps"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"모델이 로컬에 저장되었습니다: {output_dir}")

모델이 로컬에 저장되었습니다: /content/drive/MyDrive/한글리뷰/20250223_cpt_2-8_1500steps


## 9. 모델 불러오기

In [7]:
from unsloth import FastLanguageModel

output_dir = "/content/drive/MyDrive/한글리뷰/20250223_cpt_2-8_1500steps"

# FastLanguageModel로 불러오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=output_dir,  # 저장된 모델 경로
    max_seq_length=4096,  # 학습 시 사용한 max_seq_length과 동일하게 설정
    dtype=None,  # 학습 시 사용한 데이터 타입
    load_in_4bit=True  # 4bit 로드 여부
)

print("모델 로드 완료!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/164k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


모델 로드 완료!


## 10. 추론하기

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0.05, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora=False,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [10]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token =tokenizer.eos_token#"<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

In [13]:
import random
import numpy as np
import os
import torch
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(3407) # Seed 고정

In [14]:
from tqdm import tqdm
import re
test_result = []
FastLanguageModel.for_inference(model) # for_inference는 학습을 비활성화시켜서 추론 속도를 2배 빠르게 만듦
number = 0
for test_i in tqdm(test["input"]):
  # 모델 테스트 모듈
  number += 1
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          f"""{test_i}""", # 질문
          "",                                              # 생성된 답변칸 비어두기
      )
  ], return_tensors = "pt").to("cuda") # PyTorch 텐서, GPU사용

  outputs = model.generate(**inputs,            # 이전 입력값
                           max_new_tokens = 1096, # 최대 64개의 단어
                           stopping_criteria=stopping_criteria)    # 캐싱을 사용하여 이전 결과 저장

  t =tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]
  #print(t)
  match = re.search(r'Response:\n\s*(.*)', t, re.DOTALL)
  if match:
    output_text = match.group(1).strip()
    #print(f"[0]:{output_text}")
  else:
    #print(f"[1]:{t}")
    output_text = test_i
  print(f"{number} : {output_text}")
  test_result.append(output_text)


  0%|          | 1/1689 [00:03<1:29:49,  3.19s/it]

1 : 너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다! ㅎㅎ 다음에 또 올 것 같아요.


  0%|          | 2/1689 [00:06<1:24:22,  3.00s/it]

2 : 프론트가 없고, 조식도 없으며, 일반 입주민들이 사이사이에 있어 호텔처럼 관리가 잘 안 되는 느낌이네요. 가성비가 떨어져요.


  0%|          | 3/1689 [00:22<4:15:37,  9.10s/it]

...

1687 : 에만하면 다시 안 올 예정입니다. 겨울 외풍이 있습니다. 싫고. 이건 뭐, 관리할 마음이 없다는 거지.


100%|█████████▉| 1688/1689 [3:32:40<00:07,  7.27s/it]

1688 : 도착 후 크게 기대하지 않았는데 직원분들이 친절하시고 2박 머무는 동안 매일 객실도 다 청소해 주셨습니다. 아침 조식도 괜찮아서 그런지 식사 시간이 길어졌어요... 좋아요!


100%|██████████| 1689/1689 [3:33:00<00:00,  7.57s/it]

1689 : 친구들이랑 멀티플렉스룸에서 투숙했는데 넓기도 넓고 게임할 것도 많아서 좋았어요. 친구들이랑 밤새 디텐도 스위치 하는 게 재미있었어요. 게임해서 내기하면서 하루 종일 놀았어요. 핀볼이 있는데 속이 너무 비어 있어서 아기는 힘들었고, 화장실 세면대 2개가 있고 욕조, 샤워실이 구분되어 있어서 좋더라고요. 모두가 모이긴 좋은 것일까 침실, 알바룸이 있어서 넓은 공간에서 쾌적하게 지냈습니다. 다섯 명이라 두 명은 스탠다드 트윈룸, 세 명은 멀티플렉스룸에서 잤는데, 다섯 명은 멀티플렉스룸에서 자도 무방할 만큼 침대와 공간이 충분했어요. 방 가까이 붙어 달라고 했는데 옆부분으로 잘 붙여 주셔서 감사합니다~ 딱 3시 체크인이라 체크인 전에 줄이 길고 사람이 몰려서 앉을 자리가 부족하고 좀 오래 걸리는 편이라 투숙 과정이 아쉬웠는데 이후부터는 다 좋았습니다. 다음에 또 멀티플렉스 하러 가려구요><


## 11. 제출 csv만들기

In [15]:
submission = pd.read_csv('./extracted_files/sample_submission.csv', encoding = 'utf-8-sig')

In [16]:
submission['output'] = test_result

In [17]:
submission

,ID,output
0,TEST_0000,너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다...
1,TEST_0001,"프론트가 없고, 조식도 없으며, 일반 입주민들이 사이사이에 있어 호텔처럼 관리가 잘..."
2,TEST_0002,진짜 불친절해요. 살면서 머물렀던 호텔 중에 최악이었습니다. 직원인지 사장인지 체크...
3,TEST_0003,뷰 맛집~~ 그런데 방음이 미흡하네요. 층간 소음과 발코니가 이중창이 아니라서 밤에...
4,TEST_0004,방 상태는 진짜 폐헐 직전인데 전망은 좋아요. 보일러가 아주 잔잔하게 돌아서 추웠어...
...,...,...
1684,TEST_1684,"일반실 5만원이고, 프리미엄 6만원짜리는 스타일러스, 안마기도 있고, PC도 있지만..."
1685,TEST_1685,"일단 방은 이쁘고, 배달 음식은 금지어서 (외인지는 의문..) 룸서비스랑 조식으로 ..."
1686,TEST_1686,"에만하면 다시 안 올 예정입니다. 겨울 외풍이 있습니다. 싫고. 이건 뭐, 관리할 ..."
1687,TEST_1687,도착 후 크게 기대하지 않았는데 직원분들이 친절하시고 2박 머무는 동안 매일 객실도...


In [18]:
submission.to_csv('/content/drive/MyDrive/한글리뷰/mistral_20250223_cpt_2-8_1500steps_ori.csv', index = False, encoding = 'utf-8-sig')